# CS295/395: Secure Distributed Computation
## Homework 1

In [3]:
# Useful imports and utility functions

import galois
GF = galois.GF(2**31-1)

def make_additive_shares(n, x):
    """Create additive secret shares of the input, so that the shares look random but add up to the input.
    n: number of shares to create
    x: input value"""
    first_shares = GF.Random(n-1)
    last_share = x - first_shares.sum()
    return GF(list(first_shares) + [last_share])

## Question 1 (20 points)

Imagine you (the "client") want to periodically submit encrypted numbers to two "servers" for storage. After some time, you'd like to have the servers add the numbers up and return the encrypted sum, so that you don't need to compute the sum yourself.

Implement a system for *outsourced computation* of the sum of a list of numbers to solve this problem by filling in the functions below. Your solution should simulate who holds what data by updating the `server_state` and `client_state` lists.

You can assume that at least one server is **honest**, and the other is **semi-honest**. You don't know which is which ahead of time. Your solution should prevent the semi-honest server from learning any of the client's inputs.

In [10]:
# Initialize server and client state
server1_state = []
server2_state = []

# Functions to simulate sending values to each of the servers
def send_to_server1(x):
    server1_state.append(x)

def send_to_server2(x):
    server2_state.append(x)

def init():
    '''Initialize the server and client state'''
    global server1_state
    global server2_state
    global client_state

    server1_state = []
    server2_state = []

def store_number(x):
    '''Store a new input number x.'''
    s1, s2 = make_additive_shares(2, GF(x))

    send_to_server1(s1)
    send_to_server2(s2)
    
def run_server1_computation():
    '''Run the server-side computation for server 1, returning its result.'''
    return GF(server1_state).sum()
    
def run_server2_computation():
    '''Run the server-side computation for server 2, returning its result.'''
    return GF(server2_state).sum()


def decrypt_result(server1_result, server2_result):
    '''Run the client-side decryption process, returning the plaintext result of the
    server-side computation.'''
    
    return (server1_result + server2_result)

In [11]:
# TEST CASE for problem 1

# A test list of numbers
number_stream = [3,4,5,1,2,3,5,1,2,3,5,5,5,2,3,1,1,5,3,4,5]

# Initialize client & server state
init()

# Handle each number in the stream
for n in number_stream:
    store_number(n)

# Run the server computation and return it to the client
server1_result = run_server1_computation()
server2_result = run_server2_computation()

# Decrypt the result of the server computation
decrypted_result = decrypt_result(server1_result, server2_result)

# It should be the same as the sum of the original number stream
print(decrypted_result)
print(sum(number_stream))
assert decrypted_result == sum(number_stream)

68
68


## Question 2 (5 points)

Why do we choose $p = 2^{31}-1$? Does it matter that $p$ is prime? What could happen if we choose $p$ too small, and the stream of numbers ends up being very large?

p must be prime so that the field is a Galois field and every element has a multiplicative inverse. This is mandatory when using the `multFE` method because it garuntees that division is available to us.

The size of the possible secret is bound by the size of prime p. If you tried to encode something larger than p, because we are in a finite modular field it will "wrap around". There are a finite number of possible secrets which is equal to the number of elements in the finite field which is equal to p.

## Question 3 (5 points)

This system handles only unsigned (non-negative) integers. In 3-5 sentences, describe how we could extend it to handle *signed* integers?

If you can successfully encode a number from 0 to n, you can remap that space to (-n/2) to (n/2) by adding (n/2) before encoding and subtracting (n/2) after decoding. With the post/pre processing pattern you can not only get negative numbers, but you can get decimals of any resoltion by multiplying by 10^d pre-encoding and dividing by 10^d post encoding. This does not change the integrity of the protocol at all.